<a href="https://colab.research.google.com/github/Temerius/ML/blob/main/Xena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets transformers pandas matplotlib tqdm --upgrade --quiet
!pip install googletrans==4.0.0-rc1

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import datasets
from transformers import pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from googletrans import Translator


In [4]:
import json
import datetime

In [5]:
with open("gdrive/MyDrive/res.json", "r") as my_file:
    tg = my_file.read()

tg = json.loads(tg)

In [6]:
start_str_date = "2022-09-02"
end_str_date = "2024-03-21"

In [7]:
start_date = datetime.datetime.strptime(start_str_date, "%Y-%m-%d")
end_date = datetime.datetime.strptime(end_str_date, "%Y-%m-%d")
#end_date = start_date + datetime.timedelta(days=5)
formatted_date = end_date.strftime('%Y-%m-%d')
formatted_date

'2024-03-21'

In [8]:
def create_date_keys():
    date_keys = []
    current_date = start_date
    while current_date <= end_date:
        str_date = current_date.strftime('%Y-%m-%d')
        while str_date not in tg:
            current_date += datetime.timedelta(days=1)
            str_date = current_date.strftime('%Y-%m-%d')
        date_keys.append(str_date)
        current_date += datetime.timedelta(days=1)
    return date_keys

In [9]:
date_keys = create_date_keys()
print(len(date_keys), len(tg))

521 521


In [10]:
#tg[start_str_date]

In [11]:
split_name = "test"
dataset_name, dataset_config_name = "go_emotions", "simplified"
dataset_dict = datasets.load_dataset(dataset_name, dataset_config_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [95]:
#dataset_dict[split_name][0]
#dataset_dict[split_name]["text"][:100]

In [96]:
#dataset_dict['test'].features

In [97]:
#type(dataset_dict[split_name]["text"])

In [12]:
translator = Translator()

In [13]:
day_array = []

In [14]:
for i in tqdm(tg[start_str_date]):
    day_array.append(translator.translate(i, dest="en").text)

  4%|▍         | 7/173 [00:02<00:59,  2.78it/s]


KeyboardInterrupt: 

In [18]:
len(day_array)

173

In [28]:
#translated_text = translator.translate(tg[formatted_date], dest="en").text
#print(translated_text)

In [24]:
#tmp = pd.DataFrame(dataset_dict['test'])
#indexes = tmp.iloc[:,2]
#dataset_pd = pd.DataFrame(dataset_dict['test'], index=indexes)
#dataset_pd = dataset_pd.iloc[:,:2]
#dataset_pd

In [13]:
#dataset_pd.iloc[0,0]

'I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!'

In [25]:
#dataset_pd.describe()

In [15]:
labels = dataset_dict[split_name].features["labels"].feature.names
labels
#print({i: l for i, l in enumerate(labels)})

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [102]:
#num_items, num_labels = len(dataset_dict[split_name]), len(labels)
#print(num_items, num_labels)

In [103]:
#y_targets_all = np.zeros((num_items, num_labels), dtype=int)
#for i, labels_indices in enumerate(dataset_dict[split_name]["labels"]):
#    for label_index in labels_indices:
#        y_targets_all[i, label_index] = 1
#print(y_targets_all[0:3])

In [16]:
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [105]:
#model_outputs = classifier(dataset_dict[split_name]["text"])
#print(dataset_dict[split_name]["text"][0])
#print(model_outputs[0])

In [106]:
#print(dataset_dict[split_name]["text"][0])
#print(model_outputs[:3])

In [17]:
def calculate_statistic_per_day(data):
    new_data = dict(zip(labels, len(labels) * [0]))
    #print(new_data)
    for text, dictionaries in data:
        for d in dictionaries:
            new_data[d["label"]] += d["score"]
    for key in new_data.keys():
        new_data[key] /= len(data)
    return new_data

In [18]:
model_outputs = []
for i in tqdm(day_array):
    model_outputs.append((i,*classifier(i)))

100%|██████████| 7/7 [00:02<00:00,  3.30it/s]


In [19]:
day_stat = calculate_statistic_per_day(model_outputs)
day_stat

{'admiration': 0.004092953916239951,
 'amusement': 0.002300308152501072,
 'anger': 0.21117060738781998,
 'annoyance': 0.0443924725654402,
 'approval': 0.07882382035521525,
 'caring': 0.0015907060587778687,
 'confusion': 0.025501111755147576,
 'curiosity': 0.09330693301412143,
 'desire': 0.0013775582525080868,
 'disappointment': 0.003521190274373761,
 'disapproval': 0.0051718211200620446,
 'disgust': 0.006961721619258502,
 'embarrassment': 0.0009363759683245527,
 'excitement': 0.0031504108420839266,
 'fear': 0.0013659798590067243,
 'gratitude': 0.001138667600961136,
 'grief': 0.0004441801699743207,
 'joy': 0.0020953017436633153,
 'love': 0.002022236369417182,
 'nervousness': 0.00037067084797724547,
 'optimism': 0.0037552035313897897,
 'pride': 0.0003841631467886535,
 'realization': 0.008130296798689025,
 'relief': 0.00039696901720682424,
 'remorse': 0.00040916552305653956,
 'sadness': 0.0023907204408065547,
 'surprise': 0.002897064351210637,
 'neutral': 0.5786240952355521}

In [20]:
df = pd.DataFrame(day_stat, index=[start_str_date])
df

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
2022-09-02,0.004093,0.0023,0.211171,0.044392,0.078824,0.001591,0.025501,0.093307,0.001378,0.003521,...,0.002022,0.000371,0.003755,0.000384,0.00813,0.000397,0.000409,0.002391,0.002897,0.578624


In [21]:
#model_outputs = []
#for i in tqdm(dataset_dict[split_name]["text"][:100]):
#    model_outputs.append(classifier(i))

In [22]:
#model_outputs

In [25]:
for date in tqdm(date_keys):
    #print(f'|||{date}||| \n\n')
    day_array = []
    for i in tg[date]:
        day_array.append(translator.translate(i, dest="en").text)
    model_outputs = []
    for i in day_array:
        model_outputs.append((i,*classifier(i)))
    day_stat = calculate_statistic_per_day(model_outputs)
    df.loc[date] = day_stat

  0%|          | 2/521 [00:41<3:01:36, 20.99s/it]


AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [24]:
df

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
2022-09-02,0.028657,0.021386,0.037347,0.022792,0.083848,0.007694,0.050009,0.057233,0.005700,0.005756,...,0.002615,0.000706,0.010078,0.003212,0.017923,0.001268,0.000649,0.003156,0.004486,0.684748
2022-09-03,0.005581,0.050957,0.009353,0.020178,0.052340,0.007822,0.023175,0.039047,0.001906,0.008099,...,0.003441,0.000758,0.005066,0.000656,0.015150,0.002063,0.008772,0.017891,0.009878,0.680429
2022-09-04,0.012404,0.032008,0.039663,0.023963,0.081091,0.011638,0.052541,0.079826,0.011082,0.012555,...,0.006184,0.000716,0.007891,0.000603,0.016970,0.000992,0.009122,0.017201,0.002339,0.627221
